In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 119kB/s 
     |████████████████████████████████| 6.8MB 44.4MB/s 
     |████████████████████████████████| 983kB 33.2MB/s 
     |████████████████████████████████| 266kB 47.3MB/s 
     |████████████████████████████████| 1.9MB 36.2MB/s 
     |████████████████████████████████| 1.2MB 38.0MB/s 
     |████████████████████████████████| 471kB 41.0MB/s 
     |████████████████████████████████| 901kB 32.1MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282380 sha256=dbee48a192057c5f06d1c386dafcd2aa82dbc0400ea506a56711e1b26db6944d
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=653643ba08545ca0f27002ecc9890668ec6d7d1671a2c1afa3cc27104924115b
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [3]:
import ktrain


In [4]:
from ktrain import text


In [5]:
train=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/train.csv')

In [6]:
test=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/test.csv')

In [7]:
valid=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/val.csv')

In [8]:
MODEL = "/content/bert-base-qarib/"

In [9]:
def print_evaluation(gold_labels, predicted_labels):
    ## computes overall scores (accuracy, f1, recall, precision)
    accuracy = accuracy_score(gold_labels, predicted_labels) * 100
    f1 = f1_score(gold_labels, predicted_labels, average = "macro") * 100
    recall = recall_score(gold_labels, predicted_labels, average = "macro") * 100
    precision = precision_score(gold_labels, predicted_labels, average = "macro") * 100
    a = [accuracy, precision, recall, f1]
    return a

In [10]:
categories = ['neutral', 'negative', 'positive'] #classes present in the data


In [11]:
train.dropna(inplace=True)

In [12]:
valid.dropna(inplace=True)

In [13]:
test.dropna(inplace=True)

In [14]:
t = text.Transformer('qarib/bert-base-qarib', maxlen=512, class_names=categories)

In [15]:
trn = t.preprocess_train(train['cleaned_text'].to_numpy(), train['Class_camel'].to_numpy())
val = t.preprocess_test(valid['cleaned_text'].to_numpy(), valid['Class_camel'].to_numpy())




preprocessing train...
language: ar
train sequence lengths:
	mean : 11
	95percentile : 22
	99percentile : 36


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: ar
test sequence lengths:
	mean : 11
	95percentile : 22
	99percentile : 39


In [16]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=10)
# train for 3 epochs
learner.fit_onecycle(8e-5, 3)
predictor = ktrain.get_predictor(learner.model, preproc=t)

404 Client Error: Not Found for url: https://huggingface.co/qarib/bert-base-qarib/resolve/main/tf_model.h5





begin training using onecycle policy with max lr of 8e-05...
Epoch 1/3
2903/2903 [==============================] - 3460s 1s/step - loss: 0.4894 - accuracy: 0.8047 - val_loss: 0.4503 - val_accuracy: 0.8343
Epoch 2/3
2903/2903 [==============================] - 3446s 1s/step - loss: 0.3301 - accuracy: 0.8796 - val_loss: 0.4414 - val_accuracy: 0.8510
Epoch 3/3
2903/2903 [==============================] - 3448s 1s/step - loss: 0.1000 - accuracy: 0.9673 - val_loss: 0.4610 - val_accuracy: 0.8584


In [19]:
predicted_probas = predictor.predict(test['cleaned_text'].to_numpy())



In [32]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [33]:
print(print_statistics(test['Class_camel'].to_numpy(), predicted_probas))

Accuracy: 0.859
Precision: 0.858
Recall: 0.859
F_score: 0.858

              precision    recall  f1-score   support

    negative       0.84      0.85      0.85      5607
     neutral       0.81      0.77      0.79      2266
    positive       0.88      0.89      0.89      7761

    accuracy                           0.86     15634
   macro avg       0.85      0.84      0.84     15634
weighted avg       0.86      0.86      0.86     15634

(0.8585134962261737, 0.8580780596776765, 0.8585134962261737, 0.8582203746005775)


In [37]:
if not os.path.exists('/content/drive/MyDrive/Omdena_sentiment/Saved_models/QAIR/'):
  os.mkdir('/content/drive/MyDrive/Omdena_sentiment/Saved_models/QAIR/')


In [39]:
predictor._save_model('/content/drive/MyDrive/Omdena_sentiment/Saved_models/QAIR/')